In [ ]:
import numpy as np
from ice_interp import ice_interp_mod

# Define input values
lon = np.float64(255.070312500000)  # Interpolation point longitude (degrees)
lat = np.float64(74.4735717773438)  # Interpolation point latitude (degrees)

x_corners = np.array([253.978338312172, 254.698325218660, 256.152459376863, 255.464594303337], dtype=np.float64)
y_corners = np.array([74.4775053981321, 74.2803767364551, 74.6534788353970, 74.8496146897753], dtype=np.float64)
p = np.array([0.04222093679422526, 0.02939444349332336, 0.02361927128232367, 0.02457275269829678], dtype=np.float64)  # Values at the quadrilateral corners

# Prepare output variable
expected_obs = np.zeros(1, dtype=np.float64)

# Call the Fortran subroutine
expected_obs = ice_interp_mod.quad_idw_interp_f2py(
    lon, lat, x_corners, y_corners, p
)

# Print the result
print(f"Interpolated value at ({lon}, {lat}): {expected_obs}")

Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
